##Tiny NeRF
This is a simplied version of the method presented in *NeRF: Representing Scenes as Neural Radiance Fields for View Synthesis*

[Project Website](http://www.matthewtancik.com/nerf)

[arXiv Paper](https://arxiv.org/abs/2003.08934)

[Full Code](github.com/bmild/nerf)

Components not included in the notebook
*   5D input including view directions
*   Hierarchical Sampling



In [50]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

# Removed the line that tries to force TensorFlow 1.x
#if IN_COLAB:
#    %tensorflow_version 1.x

#Import tensorflow
import tensorflow as tf
#Enable eager execution for TensorFlow 2.x
tf.compat.v1.enable_eager_execution()

from tqdm import tqdm_notebook as tqdm
import numpy as np
import matplotlib.pyplot as plt

In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
# Check Tinynerf folder
tinynerf_dir = '/content/drive/MyDrive/DESMAI_TALIYAJOSEPH_2025/Tinynerf'
print("Contents of Tinynerf directory:")
!ls {tinynerf_dir}

Contents of Tinynerf directory:
Blenderllff_TF	tiny_nerf.ipynb


In [53]:
from google.colab import drive
import os
import numpy as np
from PIL import Image



# Specify Image Folder Path
image_folder = r"/content/drive/MyDrive/DESMAI_TALIYAJOSEPH_2025/Tinynerf/Blenderllff_TF"

# Debugging: Check if the folder exists
if not os.path.exists(image_folder):
    print("The specified folder does not exist.")
else:
    print("Folder exists.")

# Load and Process Images
print(f"Image folder path: '{image_folder}' Length: {len(image_folder)}")  # Debugging output
all_files = os.listdir(image_folder)
print("All files in the folder:", all_files)

# Filter for PNG files
image_filenames = [f for f in all_files if f.endswith('.png')]
print("Number of PNG images found:", len(image_filenames))

images = []
for filename in image_filenames:
    img_path = os.path.join(image_folder, filename)
    try:
        img = Image.open(img_path).convert("RGB")
        images.append(np.array(img))
    except Exception as e:
        print(f"Error loading {img_path}: {e}")

if images:
    images = np.stack(images, axis=0)
    print("Images shape:", images.shape)
else:
    print("No valid images loaded.")

# Camera Poses and Focal Length
poses = np.random.rand(100, 4, 4)  # Replace with your actual poses
focal = 138.8892067803277

# Save as .npz File
output_path = r"/content/drive/MyDrive/DESMAI_TALIYAJOSEPH_2025/my_custom_dataset.npz"
np.savez_compressed(output_path, images=images, poses=poses, focal=focal)

print("Dataset saved as my_custom_dataset.npz in", output_path)

OSError: [Errno 107] Transport endpoint is not connected: '/content/drive'

In [ ]:
import numpy as np
import os

# Specify the path to your custom dataset
dataset_path = r"/content/drive/MyDrive/DESMAI_TALIYAJOSEPH_2025/my_custom_dataset.npz"

# Check if the file exists
if not os.path.exists(dataset_path):
    print("Custom dataset does not exist at the specified path.")
else:
    print("Loading custom dataset...")

    # Load the NPZ file
    data = np.load(dataset_path)

    # Access the arrays stored in the NPZ file
    images = data['images']
    poses = data['poses']
    focal = data['focal']

    print("Images shape:", images.shape)
    print("Poses shape:", poses.shape)
    print("Focal length:", focal)

In [ ]:
#import os
#if not os.path.exists('tiny_nerf_data.npz'):
    #!wget http://cseweb.ucsd.edu/~viscomp/projects/LF/papers/ECCV20/nerf/tiny_nerf_data.npz

In [ ]:
#import os
#import numpy as np

#if not os.path.exists('tiny_nerf_data.npz'):
    #!wget http://cseweb.ucsd.edu/~viscomp/projects/LF/papers/ECCV20/nerf/tiny_nerf_data.npz

# Now you can load the data
#data = np.load('tiny_nerf_data.npz')

In [ ]:
import matplotlib.pyplot as plt

# Access images, poses, and focal length from the loaded data
images = data['images']
poses = data['poses']
focal = data['focal']

# Print information about the dataset
print("Image shape:", images.shape)  # Shape of the images array
print("Poses shape:", poses.shape)    # Shape of the poses array
print("Focal length:", focal)         # Value of the focal length

# Display a few sample images from the dataset
num_images_to_display = 5  # Number of images to display

for i in range(num_images_to_display):
    plt.figure()
    plt.imshow(images[i])
    plt.title(f"Image {i + 1}")
    plt.axis('off')  # Turn off axis labels
    plt.show()

# Load Input Images and Poses

In [ ]:
# Specify the path to your custom dataset
dataset_path = r"/content/drive/MyDrive/DESMAI_TALIYAJOSEPH_2025/my_custom_dataset.npz"  # Corrected path

# Load the NPZ file using the specified path
data = np.load(dataset_path)
)
images = data['images']
poses = data['poses']
focal = data['focal']
H, W = images.shape[1:3]
print(images.shape, poses.shape, focal)

testimg, testpose = images[101], poses[101]
images = images[:100,...,:3]
poses = poses[:100]

plt.imshow(testimg)
plt.show()

# Optimize NeRF

In [ ]:


def posenc(x):
  rets = [x]
  for i in range(L_embed):
    for fn in [tf.sin, tf.cos]:
      rets.append(fn(2.**i * x))
  return tf.concat(rets, -1)

L_embed = 6
embed_fn = posenc
# L_embed = 0
# embed_fn = tf.identity

def init_model(D=8, W=256):
    relu = tf.keras.layers.ReLU()
    dense = lambda W=W, act=relu : tf.keras.layers.Dense(W, activation=act)

    inputs = tf.keras.Input(shape=(3 + 3*2*L_embed,))
    outputs = inputs
    for i in range(D):
        outputs = dense()(outputs)
        if i%4==0 and i>0:
            outputs = tf.keras.layers.concatenate([outputs, inputs], axis=-1)
    outputs = dense(4, act=None)(outputs)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model


def get_rays(H, W, focal, c2w):
    i, j = tf.meshgrid(tf.range(W, dtype=tf.float32), tf.range(H, dtype=tf.float32), indexing='xy')
    dirs = tf.stack([(i-W*.5)/focal, -(j-H*.5)/focal, -tf.ones_like(i)], -1)
    rays_d = tf.reduce_sum(dirs[..., np.newaxis, :] * c2w[:3,:3], -1)
    rays_o = tf.broadcast_to(c2w[:3,-1], tf.shape(rays_d))
    return rays_o, rays_d



def render_rays(network_fn, rays_o, rays_d, near, far, N_samples, rand=False):

    def batchify(fn, chunk=1024*32):
        return lambda inputs : tf.concat([fn(inputs[i:i+chunk]) for i in range(0, inputs.shape[0], chunk)], 0)

    # Compute 3D query points
    z_vals = tf.linspace(near, far, N_samples)
    if rand:
      z_vals += tf.random.uniform(list(rays_o.shape[:-1]) + [N_samples]) * (far-near)/N_samples
    pts = rays_o[...,None,:] + rays_d[...,None,:] * z_vals[...,:,None]

    # Run network
    pts_flat = tf.reshape(pts, [-1,3])
    pts_flat = embed_fn(pts_flat)
    raw = batchify(network_fn)(pts_flat)
    raw = tf.reshape(raw, list(pts.shape[:-1]) + [4])

    # Compute opacities and colors
    sigma_a = tf.nn.relu(raw[...,3])
    rgb = tf.math.sigmoid(raw[...,:3])

    # Do volume rendering
    dists = tf.concat([z_vals[..., 1:] - z_vals[..., :-1], tf.broadcast_to([1e10], z_vals[...,:1].shape)], -1)
    alpha = 1.-tf.exp(-sigma_a * dists)
    weights = alpha * tf.math.cumprod(1.-alpha + 1e-10, -1, exclusive=True)

    rgb_map = tf.reduce_sum(weights[...,None] * rgb, -2)
    depth_map = tf.reduce_sum(weights * z_vals, -1)
    acc_map = tf.reduce_sum(weights, -1)

    return rgb_map, depth_map, acc_map

Here we optimize the model. We plot a rendered holdout view and its PSNR every 50 iterations.

In [ ]:
model = init_model()
optimizer = tf.keras.optimizers.Adam(5e-4)

N_samples = 64
N_iters = 1000
psnrs = []
iternums = []
i_plot = 25

import time
t = time.time()
for i in range(N_iters+1):

    img_i = np.random.randint(images.shape[0])
    target = images[img_i]
    pose = poses[img_i]
    rays_o, rays_d = get_rays(H, W, focal, pose)
    with tf.GradientTape() as tape:
        rgb, depth, acc = render_rays(model, rays_o, rays_d, near=2., far=6., N_samples=N_samples, rand=True)
        loss = tf.reduce_mean(tf.square(rgb - target))
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    if i%i_plot==0:
        print(i, (time.time() - t) / i_plot, 'secs per iter')
        t = time.time()

        # Render the holdout view for logging
        rays_o, rays_d = get_rays(H, W, focal, testpose)
        rgb, depth, acc = render_rays(model, rays_o, rays_d, near=2., far=6., N_samples=N_samples)
        loss = tf.reduce_mean(tf.square(rgb - testimg))
        psnr = -10. * tf.math.log(loss) / tf.math.log(10.)

        psnrs.append(psnr.numpy())
        iternums.append(i)

        plt.figure(figsize=(10,4))
        plt.subplot(121)
        plt.imshow(rgb)
        plt.title(f'Iteration: {i}')
        plt.subplot(122)
        plt.plot(iternums, psnrs)
        plt.title('PSNR')
        plt.show()

print('Done')

# Interactive Visualization

In [ ]:
import tensorflow as tf
import numpy as np
%matplotlib inline
from ipywidgets import interactive, widgets


trans_t = lambda t : tf.convert_to_tensor([
    [1,0,0,0],
    [0,1,0,0],
    [0,0,1,t],
    [0,0,0,1],
], dtype=tf.float32)

rot_phi = lambda phi : tf.convert_to_tensor([
    [1,0,0,0],
    [0,tf.cos(phi),-tf.sin(phi),0],
    [0,tf.sin(phi), tf.cos(phi),0],
    [0,0,0,1],
], dtype=tf.float32)

rot_theta = lambda th : tf.convert_to_tensor([
    [tf.cos(th),0,-tf.sin(th),0],
    [0,1,0,0],
    [tf.sin(th),0, tf.cos(th),0],
    [0,0,0,1],
], dtype=tf.float32)


def pose_spherical(theta, phi, radius):
    c2w = trans_t(radius)
    c2w = rot_phi(phi/180.*np.pi) @ c2w
    c2w = rot_theta(theta/180.*np.pi) @ c2w
    c2w = np.array([[-1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]]) @ c2w
    return c2w


def f(**kwargs):
    c2w = pose_spherical(**kwargs)
    rays_o, rays_d = get_rays(H, W, focal, c2w[:3,:4])
    rgb, depth, acc = render_rays(model, rays_o, rays_d, near=2., far=6., N_samples=N_samples)
    img = np.clip(rgb,0,1)

    plt.figure(2, figsize=(20,6))
    plt.imshow(img)
    plt.show()


sldr = lambda v, mi, ma: widgets.FloatSlider(
    value=v,
    min=mi,
    max=ma,
    step=.01,
)

names = [
    ['theta', [100., 0., 360]],
    ['phi', [-30., -90, 0]],
    ['radius', [4., 3., 5.]],
]

interactive_plot = interactive(f, **{s[0] : sldr(*s[1]) for s in names})
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

# Render 360 Video

In [ ]:
frames = []
for th in tqdm(np.linspace(0., 360., 120, endpoint=False)):
    c2w = pose_spherical(th, -30., 4.)
    rays_o, rays_d = get_rays(H, W, focal, c2w[:3,:4])
    rgb, depth, acc = render_rays(model, rays_o, rays_d, near=2., far=6., N_samples=N_samples)
    frames.append((255*np.clip(rgb,0,1)).astype(np.uint8))

import imageio
f = 'video.mp4'
imageio.mimwrite(f, frames, fps=30, quality=7)

In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('video.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls autoplay loop>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)